In [2]:
# !pip install openpyxl
# 필요 import 한번에 
from bs4 import BeautifulSoup as BS
from tqdm import tqdm 
import numpy as np
import os
import pandas as pd
import requests
import pickle
import time

In [172]:
# 엑셀 파일을 읽어옵니다.
directory_path = "../../../Downloads"  # 다운로드 폴더에 있을경우
# 엑셀 파일들의 이름 리스트 ('_통계' 문구 있는것만)
excel_files = [f for f in os.listdir(directory_path) if f.endswith('_통계.xlsx')]

# 각 파일의 시트를 저장할 딕셔너리
all_data = {}

for file_name in excel_files:
    file_path = os.path.join(directory_path, file_name)
    # 파일에서 모든 시트를 읽어옴
    sheets = pd.read_excel(file_path, sheet_name=None)
    
    # 각 파일의 시트들을 딕셔너리에 저장
    all_data[file_name] = sheets

# 결과 확인 (파일과 시트 이름을 출력)
for file, sheets in all_data.items():
    print(f"File: {file}")
    for sheet_name, df in sheets.items():
        print(f"  Sheet name: {sheet_name}")
        print(df.head())  # 시트 데이터의 첫 몇 줄을 출력

File: 2023년_10월_자동차_등록자료_통계.xlsx
  Sheet name: 01.통계표
  Unnamed: 0 Unnamed: 1 Unnamed: 2        <     자동차      통계표 Unnamed: 6  \
0      조회년월:    2023.10        NaN      NaN     NaN      NaN        NaN   
1         차종         차종         승용      NaN     NaN      NaN         승합   
2        용도별        NaN         관용      자가용     영업용        계         관용   
3        시도별        NaN        NaN      NaN     NaN      NaN        NaN   
4         서울        NaN       4592  2649600  120923  2775115       3710   

       > Unnamed: 8 Unnamed: 9  ... Unnamed: 12 Unnamed: 13 Unnamed: 14  \
0    NaN        NaN        NaN  ...         NaN         NaN         NaN   
1    NaN        NaN        NaN  ...         NaN         NaN          특수   
2    자가용        영업용          계  ...         영업용           계          관용   
3    NaN        NaN        NaN  ...         NaN         NaN         NaN   
4  73036      14996      91742  ...       60266      313655         415   

  Unnamed: 15 Unnamed: 16 Unnamed: 17 Unname

In [ ]:
# 뭐뭐들었나 확인해봄
excel_files

['2023년_10월_자동차_등록자료_통계.xlsx',
 '2023년_11월_자동차_등록자료_통계.xlsx',
 '2023년_12월_자동차_등록자료_통계.xlsx',
 '2024년_01월_자동차_등록자료_통계.xlsx',
 '2024년_02월_자동차_등록자료_통계.xlsx',
 '2024년_03월_자동차_등록자료_통계.xlsx',
 '2024년_04월_자동차_등록자료_통계.xlsx',
 '2024년_05월_자동차_등록자료_통계.xlsx',
 '2024년_06월_자동차_등록자료_통계.xlsx',
 '2024년_07월_자동차_등록자료_통계.xlsx',
 '2024년_08월_자동차_등록자료_통계.xlsx',
 '2024년_09월_자동차_등록자료_통계.xlsx',
 '2024년_10월_자동차_등록자료_통계.xlsx']

In [143]:
# 시트이름 조회
list(all_data[excel_files[0]])

['01.통계표',
 '02.통계표_시군구',
 '03.수입차_시군구',
 '04.성별_연령별',
 '05.차종별_등록현황(전체)',
 '06.차종별_등록현황(관용)',
 '07.차종별_등록현황(자가용)',
 '08.차종별 등록현황(영업용)',
 '09.차종별_유형별 현황',
 '10.연료별_등록현황',
 '11.최대적재량별_등록현황',
 '12.배기별(승용차)_등록현황',
 '13.차종별_등록(시도)',
 '14.차종별_상세등록(시도)',
 '15.차령별_차종별_용도별 등록현황',
 '16.승차정원별(승합차) 등록현황',
 '17.차종별_규모별 등록현황_초소형',
 '18.자가용 자동차 증가추세',
 '19.연도별 자동차 등록현황',
 '20.신규 등록현황(당월)',
 '21.신규 등록현황(누계)',
 '22.변경 등록현황(당월)',
 '23.변경 등록현황(누계)',
 '24.이전 등록현황(당월)',
 '25.이전 등록현황(누계)',
 '26.말소 등록현황(당월)',
 '27.말소 등록현황(누계)']

In [173]:

all_df = pd.DataFrame()
for file, sheets in all_data.items():
    # 특정 시트 이름을 선택해서 변환
    year = file[:4]
    month = file[6:8]
    if '04.성별_연령별' in sheets:
        df = sheets['04.성별_연령별']
        df.columns = df.iloc[1]  # 세 번째 줄을 컬럼으로 설정
        df = df.drop([0, 1]).reset_index(drop=True)  # 첫 세 줄 제거 후 인덱스 재설정
        df.rename(columns={('연령/시도'): ('연령')}, inplace=True) # 연령/시도 -> 연령으로 컬럼 변경 
        df.drop(columns=['총계'], inplace=True) # 총계 컬럼 삭제
        # 성별 컬럼에 남성/여성으로 채움 
        df[("성별")] = df[("성별")].fillna(method='ffill')
        # 합계, 사업자 행 삭제 
        df = df[~df['연령'].isin(['계', '법인 및 사업자']) & df['연령'].notna()]
        # 성별, 연령외의 컬럼을 값으로 변경 
        df2 = df.melt(id_vars=[('성별'), ('연령')])
        # 컬럼명 변경
        df2.columns = ['gender', 'age', 'district_si', 'num']
        # 년, 월 
        df2['year'] = year
        df2['month'] = month
        # 컬럼순서 조절 
        df2 = df2[['year', 'month', 'gender', 'age', 'district_si', 'num']]
        # 결과를 all_df에 누적
        all_df = pd.concat([all_df, df2], ignore_index=True)
        

C:\Users\Playdata\AppData\Local\Temp\ipykernel_8252\1099593437.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[("성별")] = df[("성별")].fillna(method='ffill')
C:\Users\Playdata\AppData\Local\Temp\ipykernel_8252\1099593437.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[("성별")] = df[("성별")].fillna(method='ffill')
C:\Users\Playdata\AppData\Local\Temp\ipykernel_8252\1099593437.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[("성별")] = df[("성별")].fillna(method='ffill')
C:\Users\Playdata\AppData\Local\Temp\ipykernel_8252\1099593437.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[("성별")] = df[("성별")].fillna(method='

In [ ]:
# 고유한 PK로 사용할 'index' 컬럼 생성
all_df.reset_index(drop=True, inplace=True)  # 인덱스 초기화
all_df['id'] = all_df.index + 1  # 'pk' 컬럼 생성 (1부터 시작하는 고유 인덱스)

# 컬럼 순서에 'pk' 추가
all_df = all_df[['id', 'year', 'month', 'gender', 'age', 'district_si', 'num']]


,id,year,month,gender,age,district_si,num
0,1,2023,10,남성,10대이하,서울,2306
1,2,2023,10,남성,20대,서울,53299
2,3,2023,10,남성,30대,서울,318679
3,4,2023,10,남성,40대,서울,503280
4,5,2023,10,남성,50대,서울,531615
...,...,...,...,...,...,...,...
3973,3974,2024,10,여성,50대,제주,34488
3974,3975,2024,10,여성,60대,제주,22463
3975,3976,2024,10,여성,70대,제주,5058
3976,3977,2024,10,여성,80대,제주,753


In [ ]:
# pip install sqlalchemy
# !pip install mysqlclient 
# !pip install pymysql

# db에 넣기 

In [ ]:
# db 연결정보 
# 이부분 지움

In [ ]:
import pymysql
# 이부분 지움
cursor = con.cursor()

0

In [178]:
# 테이블 생성
create_table_sql = """CREATE TABLE gender_age (
            id int NOT NULL,
            year char(50) NULL,
            month char(50) NULL,
            gender char(10) NULL,
            age VARCHAR(15) NULL,
            district_si VARCHAR(20) NULL,
            num int NULL
        );"""

cursor.execute(create_table_sql)

# PRIMARY KEY 추가
alter_table_sql = "ALTER TABLE gender_age ADD PRIMARY KEY (id);"
cursor.execute(alter_table_sql)

0

In [180]:

# 테이블에 데이터 넣기 
all_df.to_sql('gender_age', if_exists='append', index=False, con=engine) 

3978

In [170]:
all_df

,pk,year,month,gender,age,district_si,num
0,1,2023,10,남성,10대이하,서울,2306
1,2,2023,10,남성,20대,서울,53299
2,3,2023,10,남성,30대,서울,318679
3,4,2023,10,남성,40대,서울,503280
4,5,2023,10,남성,50대,서울,531615
...,...,...,...,...,...,...,...
3973,3974,2024,10,여성,50대,제주,34488
3974,3975,2024,10,여성,60대,제주,22463
3975,3976,2024,10,여성,70대,제주,5058
3976,3977,2024,10,여성,80대,제주,753
